### CONFIGURATION

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
import numpy as np
import os
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [3]:
# BED FILE
USE_BED = False
LOAD_NON_BED_MUTATIONS = False
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"

# TRUTH SET
SAMPLE_NAMES_TRUTH = {'SPIKEIN':'truth'}
VCF_PATH_TRUTH = "/Users/peterpriestley/hmf/analyses/dreamChallenge/"
VCF_FILE_NAME_TRUTH = "synthetic.challenge.set3.tumor.20pctmasked.truth.vcf"

# COMBINED VCF CONFIG
#VCF_SAMPLE = "IS3.snv.indel.sv"
VCF_PATH = "/Users/peterpriestley/hmf/analyses/dreamChallenge/sv_analysis/"
VCF_FILE_NAME = "somaticSV.vcf"
#SAMPLE_NAMES = {'G15512.prenormal.sorted': 'normal', 'IS3.snv.indel.sv':'tumor'}
SAMPLE_NAMES = {'IS3.snv.indel.sv':'tumor'}

In [4]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    #if x[-10:] == "sliced.vcf":
    #    VCF_SAMPLES.append(x[40:52] + "R_" + x[40:52] + "T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf")
    if x[-13:] == "somaticSV.vcf":
        VCF_SAMPLES.append(x)
VCF_SAMPLES

['somaticSV.vcf']

### Functions

In [5]:
def calculateTruth(sv_caller,sv_type,chrom,startMin,startMax,lenMin,lenMax):
    if sv_caller == "MANTA" and sv_type == 'INS':   #offset for MANTA INS + Does not give a lenth
        startMin = startMin - 0.5 * (lenMax)
        return dfTruth[(dfTruth.chrom==chrom)&(dfTruth.truthSVStartMax>=startMin-300)&(dfTruth.truthSVStartMin<=startMax+300)]['id'].count()
    elif sv_type == 'BND' or sv_type == 'TRA':
        if sv_caller == "DELLY":
            startMin = startMin - 100
        return dfTruth[(dfTruth.chrom==chrom)&(dfTruth.truthSVStartMax>=startMin)&(dfTruth.truthSVStartMin<=startMax)]['id'].count()
    elif sv_caller == "DELLY" and (sv_type == 'DUP' or sv_type == 'INV'):
        lenMax = lenMax + 125
        lenMin = lenMin - 150
        startMax = startMax + 200
        startMin = startMin - 200
    elif sv_caller == "DELLY" and sv_type == 'DEL':
        lenMin = lenMin - 150
        startMax = startMax + 200
    return dfTruth[(dfTruth.chrom==chrom)&(dfTruth.truthSVStartMax>=startMin-300)& \
            (dfTruth.truthSVStartMin<=startMax+300)&(dfTruth.truthSVLenMax>=lenMin-300)&(dfTruth.truthSVLenMin<=lenMax+300)]['id'].count()
def calculateTruthSubVariantType(sv_caller,sv_type,chrom,startMin,startMax,lenMin,lenMax):
    if sv_caller == "MANTA" and sv_type == 'INS':   #offset for MANTA INS + Does not give a lenth
        startMin = startMin - 0.5 * (lenMax)
        startMax = startMax
    elif sv_caller == "DELLY" and sv_type == 'DEL':
        startMax = startMax + 200
    return dfTruth[(dfTruth.chrom==chrom)&(dfTruth.truthSVStartMax>=startMin)& \
            (dfTruth.truthSVStartMin<=startMax)]['variantSubType'].max() 

In [6]:
def calcuatePrecisionSensivityMatrix(df):
    outputdata = []
    variantSubTypes = df['variantSubType'].unique()
    for variantSubType in variantSubTypes:
        truePositives = len(df[(df['truth'] == True) &(df['variantSubType'] == variantSubType)])
        positives = len(df[(df['variantSubType'] == variantSubType)])
        truthSet = len(dfTruth[(dfTruth.variantType =="SV")&(dfTruth.variantSubType ==variantSubType)]) 
        falseNegatives = truthSet - truePositives
        if positives > 0:
            outputdata.append([variantSubType, truthSet,truePositives,positives-truePositives, falseNegatives, \
                                   round(truePositives/float(positives),4),round(truePositives/float(max(truthSet,1)),4)])
    
    outputDF = pd.DataFrame(outputdata)
    outputDF.columns = (['variantSubType','truthSet','truePositives','falsePositives','falseNegatives','precision','sensitivity'])
    return outputDF.sort_values(['variantSubType'])

<h3> Load VCFs and Prepare DF

In [7]:
## LOAD TRUTH SET VCF
if USE_BED:
    bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
    dfTruth = aVCF.loadVariantsFromVCF(VCF_PATH_TRUTH,VCF_FILE_NAME_TRUTH,SAMPLE_NAMES_TRUTH,"Mix-in Truth Set",True,True,bed,LOAD_NON_BED_MUTATIONS)
else:
    dfTruth = aVCF.loadVariantsFromVCF(VCF_PATH_TRUTH,VCF_FILE_NAME_TRUTH,SAMPLE_NAMES_TRUTH,"Mix-in Truth Set",True,False)
dfTruth = dfTruth[(dfTruth.variantType=="SV")&(dfTruth.variantSubType<>"IGN")&(dfTruth.variantSubType<>"MSK")]

reading vcf file: synthetic.challenge.set3.tumor.20pctmasked.truth.vcf
reading VCF File line: 1
Number variants loaded: 22266


In [8]:
dfTruth[dfTruth.variantType=='SV'][['variantType','variantSubType','ref']].groupby(['variantType','variantSubType']).agg('count')

ref
variantType variantSubType     
SV          DEL             705
            DUP             746
            INS             688
            INV             747

In [9]:
#Load all VCF_SAMPLES to DF
dfProd = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        dfProd = pd.concat([dfProd,aVCF.loadVariantsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE,True,True,bed,LOAD_NON_BED_MUTATIONS)])
    else:
        dfProd = pd.concat([dfProd,aVCF.loadVariantsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE,True,False)])
dfProd = dfProd[(dfProd.variantType=="SV")&(dfProd.variantSubType<>"IGN")]

reading vcf file: somaticSV.vcf
reading VCF File line: 1
Number variants loaded: 3096


In [10]:
dfProd[['variantType','variantSubType','ref']].groupby(['variantType','variantSubType']).agg('count')

ref
variantType variantSubType     
SV          BND             586
            DEL             552
            DUP             622
            INS             109
            INV             617

In [11]:
SV_CALLER = 'MANTA'
#SV_CALLER = 'DELLY'
dfProd['truth'] = np.vectorize(calculateTruth)(SV_CALLER,dfProd['variantSubType'],dfProd['chrom'], dfProd['tumorSVStartMin'],dfProd['tumorSVStartMax'],dfProd['tumorSVLenMin'],dfProd['tumorSVLenMax'])
dfProd['truthVariantSubType'] = np.vectorize(calculateTruthSubVariantType)(SV_CALLER,dfProd['variantSubType'],dfProd['chrom'], dfProd['tumorSVStartMin'],dfProd['tumorSVStartMax'],dfProd['tumorSVLenMin'],dfProd['tumorSVLenMax'])

### PRECISION + SENSITIVITY

In [ ]:
##### NOTES
# don't display duplicate BNDs)
# Delly INS???

In [12]:
#PRECISION + SENSITIVITY by caller - PROD
outputDF = calcuatePrecisionSensivityMatrix(dfProd)
outputDF

,variantSubType,truthSet,truePositives,falsePositives,falseNegatives,precision,sensitivity
3,BND,0,232,354,-232,0.3959,232.0000
1,DEL,705,532,20,173,0.9638,0.7546
2,DUP,746,617,5,129,0.9920,0.8271
4,INS,688,100,9,588,0.9174,0.1453
0,INV,747,611,6,136,0.9903,0.8179


In [ ]:
pd.pivot_table(dfProd[dfProd.variantType=='SV'], values='id', index=['variantSubType'], columns=['truth','truthVariantSubType'], aggfunc='count').fillna("")

In [ ]:
pd.pivot_table(dfProd[dfProd.variantType=='SV'], values='id', index=['variantSubType'], columns=['truth','truthVariantSubType'], aggfunc='count').fillna("")

In [15]:
dfProd[((dfProd.variantSubType == 'DUP'))].head(50)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,tumorallele,tumorAF,tumorDP,tumorQS,tumorSGT,tumorindelDiff,tumorSVLenMin,tumorSVLenMax,tumorSVStartMin,tumorSVStartMax,patientName,truth,truthVariantSubType
5,1,4601498,1:4601498,1.018461,MantaDUP:TANDEM:557:0:1:0:0:0,T,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.322222,1,-1.0,"61,",,4296,4298,4601498,4601499,somaticSV.vcf,1,DUP
6,1,5356241,1:5356241,1.021489,MantaDUP:TANDEM:689:0:1:0:0:0,T,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.950820,1,-1.0,"3,5",,6334,6334,5356241,5356241,somaticSV.vcf,1,DUP
16,1,20415753,1:20415753,1.081909,MantaDUP:TANDEM:2609:0:1:0:0:0,T,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.960317,1,-1.0,"5,1",,5378,5380,20415753,20415754,somaticSV.vcf,1,DUP
19,1,23438336,1:23438336,1.094035,MantaDUP:TANDEM:2928:0:1:0:0:0,C,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.139241,1,-1.0,"68,",,2494,3617,23438053,23438619,somaticSV.vcf,1,DUP
20,1,30249359,1:30249359,1.121361,MantaDUP:TANDEM:3670:0:1:0:0:0,A,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,1.000000,1,-1.0,"0,4",,7700,7704,30249359,30249361,somaticSV.vcf,1,DUP
21,1,30502649,1:30502649,1.122377,MantaDUP:TANDEM:3669:0:1:0:0:0,A,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.951613,1,-1.0,"3,5",,17983,17983,30502649,30502649,somaticSV.vcf,1,DUP
22,1,31134080,1:31134080,1.124911,MantaDUP:TANDEM:3786:0:1:0:0:0,A,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.774648,1,-1.0,"16,",,10017,10019,31134080,31134081,somaticSV.vcf,1,DUP
39,1,45141854,1:45141854,1.181110,MantaDUP:TANDEM:5379:0:1:0:0:0,G,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.987805,1,-1.0,"2,1",,3638,3638,45141854,45141854,somaticSV.vcf,1,DUP
43,1,48255654,1:48255654,1.193603,MantaDUP:TANDEM:5710:0:1:0:0:0,T,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.258427,1,-1.0,"66,",,9462,9462,48255654,48255654,somaticSV.vcf,1,DUP
44,1,48350489,1:48350489,1.193983,MantaDUP:TANDEM:5768:0:1:0:0:0,T,test,0,SV,DUP,PASS,,False,False,,,,,False,<DUP:TANDEM>,0.560606,1,-1.0,"29,",,20600,20602,48350489,48350490,somaticSV.vcf,1,DUP


In [ ]:
dfTruth[(dfTruth.chromFrac>2.65)].head(2)

### 

In [ ]:
def findMatchingBNDChromPos(df):
    df['matchingBNDChromPos'] = ""
    for i in range(0,df.shape[0]):
        if df.iloc[i]['variantSubType'] == "BND":
            tmp = df.iloc[i]['tumorallele'].find("]")
            if tmp >= 0:
                df.iloc[i,df.columns.get_loc('matchingBNDChromPos')] = df.iloc[i]['tumorallele'][tmp+1:df.iloc[i]['tumorallele'].find("]",tmp+1)]
    return df
        

In [ ]:
dfProd = findMatchingBNDChromPos(dfProd)

In [ ]:
len(dfProd[(dfProd.variantSubType=='BND')&(dfProd.matchingBNDChromPos<>'')])

In [ ]:
dfProd[dfProd.matchingBNDChromPos=='8:128753374'].truth

In [ ]:
dfProd["duplicate"] = AND (dfProd.duplicated(subset=['matchingBNDChromPos'],keep=False))

In [ ]:
pd.pivot_table(dfProd[dfProd.variantType=='SV'], values='id', index=['chrom'], columns=['truth','variantSubType'], aggfunc='count').fillna("")

In [ ]:
dfProd[(dfProd.variantSubType<>'BND')&(dfProd.truth==0)]